In [1]:

'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

/home/panshop/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


614400/600901 [==============================] - 2s 3us/step
corpus length: 600893
total chars: 57
nb sequences: 200285


In [8]:
print(sentences[:3])
#print(next_chars[0])

preface


supposing that truth is a woma
n


In [11]:
print('Vectorization... One Hot Vector')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
print(x.shape, y.shape)

Vectorization... One Hot Vector
(200285, 40, 57) (200285, 57)


In [ ]:
##mejoras:
-embedding para characteres
-tfidf para caracteres.

In [12]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)),return_sequences=False,stateful=False))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_2 (Dense)              (None, 57)                7353      
_________________________________________________________________
activation_2 (Activation)    (None, 57)                0         
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [ ]:
##mejoras:
otra LSTM entemedio
una Dense entremedio
una GRU envez
con stateful True, shuffle=False


##variantes:
cantidad de caracteres como vector

In [13]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence): #input
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char #for next character

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,batch_size=128,epochs=60, callbacks=[print_callback])

Epoch 1/60
200285/200285 [==============================] - 111s 554us/step - loss: 1.9864

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "se at all, it is a delicate and at the
s"
se at all, it is a delicate and at the
sense to the sense to the stranger and the sense to the sense to the sense and the self as in the more the strong the sense in the first and the strong and and the sense to the proble in the sense and the proble there and the sense in the strong and and therefore inder to the sense indeed to the pleter of the self-subjer to the sense to the so a of the proble to the sense to the sense of the sense 
----- diversity: 0.5
----- Generating with seed: "se at all, it is a delicate and at the
s"
se at all, it is a delicate and at the
sense to the explusent of the say are a dingle to an a searnal the sentarity with there of all the more of the ories in the more suphines, indee have and hearter in the sore light to the contrany and distrus

us, a play to seepe in the phenomeric,
that with iture what foblity especilative, and to general acre, that alcenty into something suopided thereines man-delicate e pheres. an inficed treasuration to rewore and time
reguided belost dgucener with among it natirism of philosopher and "iden reasing to who un
period uprespeds aiced, art a
----- diversity: 1.2
----- Generating with seed: "w would we then contemplate men
and thin"
w would we then contemplate men
and thing of metaphysic memite of others, as so speakour and
buusies
his divilies's suiled:
under no, irof that suppresped to live "hys, that mys fest ommaises of ciesucty, woold,"" not: whund to
him, you shoulsnnic almost tort and drursection--if the
praned.--he wisting, anysouranal lay at laterableated le most profoulsk of reason issal.! wems
and consuied awponerturs: the quite of willne,
sihe
norental

Epoch 5/60
200285/200285 [==============================] - 95s 475us/step - loss: 1.4644

----- Generating text after Epoch: 4
--

is not its equal in suffering. profound of the same things and state of the spirit and said, as the subject of the souther of the self-decision of the self-demind of the souther and the souther of the southerness of the conscience of the souther of the sentiments and souther that the state of the strength of the sention to the same the scientific superiory and all the spirit and soul and souther the same accustomed to the sentiments of 
----- diversity: 0.5
----- Generating with seed: "is not its equal in suffering. profound "
is not its equal in suffering. profound and
wiseld of
the convical and a personality and stated that it is the eronion of all the self-danger of the sentiments and staterness of the age in them is reading in the tort, that is there is there is the beleesies of all they that is and is, when there is there is also to the after taking and actually the epraped in the truths of such a things as it is not there are all
them because here, and 
----- diversity: 1.0
-----

nothing is necessary are on orderatios "hyieuver what must says carnity
were as ye justorusisiy take of 
----- diversity: 1.2
----- Generating with seed: "affinities is very hard to find; still h"
affinities is very hard to find; still have lixer time these terence, from his "lehrang! whish irser, commais prepero" with the very vang stands. it dees larg; in the flutevoug.qunishly new jayvjeoun a motion to true
tone
unrighty tact, rank be own partru and lengers in our fundamentally, he unferrstian.

1, the case oncely beutable of penrrence cay to which must waking, sitar, ama nocy
preavour.--idego, of cuisions them immorbeg of man
Epoch 12/60
200285/200285 [==============================] - 79s 393us/step - loss: 1.3714

----- Generating text after Epoch: 11
----- diversity: 0.2
----- Generating with seed: "istinguish with our feeling [that which "
istinguish with our feeling [that which is a sense of the more and assument and in the sense of the more of the soul of the more and experie

of the head, refer to his brother man is a simplition of the individual, the liet and is to an experience to use called the southernal preservination of the demonation of the same
by prehamps one wishes and open thereby sometimes and the soul of the sentiment, the ordealing of the sense of the same powers of the good religious interpreten of the person and their subject of the constitute the primitiant of as the strive and the s
----- diversity: 1.0
----- Generating with seed: "t shake
of the head, refer to his brothe"
t shake
of the head, refer to his brothery-worlds, if divised attempt conficuration oftrandrable importuriale with such accustes of
manking what in supond,
linegsuwly
pecatered dangerous learned
of
love or a mort, and about first and
exists
yain them, as the headed delight to him or. he can knownsing by constitutes one, his choint" name. as for his consequence with really entingly him who knowledge extrampule
for every form, on his shad
----- diversity: 1.2
----- Generat

...

f the sense of the only the substaterm of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of its such a sense of the sense of the sense of the sense of the same time and sense, and as it is a struggle of the sense of the sense of the sense of the sense of the same and the prese
----- diversity: 0.5
----- Generating with seed: "according to slave-morality, therefore, "
according to slave-morality, therefore, not the end is reficelonces
believe the truth of such a desire to the world in the occasional the real of the senses of the right and such a such a serious and soul
far to the lamin of the processes of the fact this morality and struggles the approtrible man, and to be it is all the hard of the last consciousness of the bottct the problem of the senses and alled--that one--and such as the sense of
----- diversity: 1.0
----- Generating with seed: "according to slave-morality, therefore, "
according to slave-morality, therefore, gea

merit, buo' transing, to hask and at the philosophy it ancient whole every spere, with rank have rank yourse,
ede
for head, this nation; not wasters,
morally interk
refiountnessly, with which authorsen woman wishes to sick, from acciuman man:
ai preven form,
that they pertained to his tingfoust obsures there and will and have accultis that,
for of
latem how ptical disof) that many naturey,
which condit
Epoch 25/60
200285/200285 [==============================] - 88s 438us/step - loss: 1.3172

----- Generating text after Epoch: 24
----- diversity: 0.2
----- Generating with seed: "late upon the russians, as above all the"
late upon the russians, as above all the strength of the sense of the presurde of the instinct of the problem of the sense of the present and all the dispose of the sense of the sense, and a sort of a man is such as the streat that the sense of the sense of the universal and standard of the sense of the sense of the present to the sense of the sense of the fact that the

to so the man that is the conscious and all the same the scientific the sense of the spirit of whose enduring the strange on the powerful extent morality and significance of the sense the senses of the significance of the last the singulate things to the possible in the such such a contradiction of extents to the strange of t
----- diversity: 1.0
----- Generating with seed: " true, there may be a metaphysical world"
 true, there may be a metaphysical worldge inthicaticiar that thre think and idant. the origin of nearer-sacririsled, one. for his hence but alone therefore one must live so, tacty once fact: the
signife of this "scurate." a things of the most o"gstrally--on the comparable in man romanged to themselves, and with wodly oneigments are thouse pured grannerable among ficuling..--know them this
contection and dog, that the
sense of goldgandi
----- diversity: 1.2
----- Generating with seed: " true, there may be a metaphysical world"
 true, there may be a metaphysical world from t

 39168/200285 [====>.........................] - ETA: 1:11 - loss: 1.2739

KeyboardInterrupt: 

### Predict on test time.

In [32]:

generated = ''
sentence = "as a mirror, that the deep heavens may !" #rellenar con un simbolo de "null" #
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

diversity=1
for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence): #input
        x_pred[0, t, char_indices[char]] = 1.

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, diversity) #not deterministic
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char #for next character

    sys.stdout.write(next_char)
    sys.stdout.flush()

----- Generating with seed: "as a mirror, that the deep heavens may !"
as a mirror, that the deep heavens may ! somein, why the
judgment and exist, and for will was found consideration for exalted that is a churhce, science, what or tejorbomen self-danghe himself for purfidentable germans in the fear of the most mit down by all ourselves to the estimate of his more for his it, here--as a new and bad given. knows to indeed and heart, in beott and of
suffering one in the genius of common, is characters of
re